In [30]:
import numpy as np
import pandas as pd
import re
import string
import pickle

In [31]:
def remove_punct(text):
    for punct in string.punctuation:
        text=text.replace(punct,'')
    return text



In [32]:
with open ('../static/model/model.pickle','rb') as f:
    model = pickle.load(f)

In [33]:
vocab = pd.read_csv('../static/model/vocabulary.txt',header=None)
tokens = vocab[0].tolist()

In [34]:
with open('../static/model/corpora/stopwords/english','r') as file:
    sw=file.read().splitlines()

In [35]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [36]:

def preprocessing(text):
    data = pd.DataFrame([text],columns=["tweet"])
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(x.lower() for x in x.split()))
    data["tweet"] = data["tweet"].apply(lambda x :" ".join(re.sub(r'^https?:\/\/.*[\r\n]*','',x,flags=re.MULTILINE) for x in x.split()))
    data["tweet"] = data["tweet"].apply(remove_punct)
    data["tweet"] =data["tweet"].str.replace(r'\d+','',regex=True)
    data["tweet"] = data["tweet"].apply(lambda x : " " .join(x for x in x.split() if x not in sw))
    data["tweet"] = data["tweet"].apply(lambda x : " " .join(ps.stem(x) for x in x.split()))
    return data["tweet"]

In [37]:
def vectorizer(ds,vocabluary):
    vectorized_list =[]
    for sentence in ds:
        sentence_lst=np.zeros(len(vocabluary))

        for i in range(len(vocabluary)):
            if vocabluary[i] in sentence.split():
                sentence_lst[i] = 1

        vectorized_list.append(sentence_lst)
    vectorized_list_new = np.asarray(vectorized_list, dtype=np.float32)
    return vectorized_list_new

In [42]:
def get_predection(vectorized_txt):
    predection = model.predict(vectorized_txt)
    if predection == 1 :
        return 'negative'
    else :
        return 'positive'

In [43]:
txt = 'good .grade awsome'
preprocessed_txt = preprocessing(txt)
vectorized_txt = vectorizer(preprocessed_txt,tokens)
prediction = get_predection(vectorized_txt)
prediction

'positive'